In [57]:
import pandas as pd
import numpy as np
import mysql.connector

In [58]:
df_uni = pd.read_pickle("../notebooks/universidades.pkl")

df_uni.head(1)

,alpha_two_code,state_province,country,name,web_pages,lat,long
0,AR,Buenos Aires,Argentina,Universidad Atlantida Argentina,http://www.atlantida.edu.ar/,-34.607568,-58.437089


In [59]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
        # nuestra clase va a recibir dos parámetros que son fijos a lo largo de toda la BBDD, el nombre de la BBDD y la contraseña con el servidor. 
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña

    # método para crear la BBDD 
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                    user="root",
                                    password=f'{self.contraseña}') 
        mycursor = mydb.cursor()

        print("La conexión se ha realizado con éxito.")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")


            print(f"La base de datos {self.nombre_bbdd} se ha creado con éxito o ya existía anteriormente.")
            
        except:

            print(f"La base de datos {self.nombre_bbdd} no se ha podido crear.")


    def crear_tablas(self):
        mydb = mysql.connector.connect(host="localhost",
                                    user="root",
                                    password=f'{self.contraseña}') 
        mycursor = mydb.cursor()


        query1 = f"""CREATE TABLE IF NOT EXISTS `{self.nombre_bbdd}`.`países` (
                    `idestado` INT NOT NULL AUTO_INCREMENT,
                    `nombre_país` VARCHAR(45) NOT NULL,
                    `nombre_provincia` VARCHAR(45) NOT NULL,
                    `latitud` DECIMAL (10, 7) NULL,
                    `longitud` DECIMAL (10, 7) NULL,
                    PRIMARY KEY (`idestado`));"""
        
        query2 = f"""CREATE TABLE IF NOT EXISTS `{self.nombre_bbdd}`.`universidades` (
                    `id_universidad` INT NOT NULL AUTO_INCREMENT,
                    `nombre_universidad` VARCHAR(100) NULL,
                    `pagina_web` VARCHAR(100) NULL,
                    `países_idestado` INT NOT NULL,
                    PRIMARY KEY (`id_universidad`),
                    INDEX `fk_universidades_países_idx` (`países_idestado` ASC) VISIBLE,
                    CONSTRAINT `fk_universidades_países`
                        FOREIGN KEY (`países_idestado`)
                        REFERENCES `{self.nombre_bbdd}`.`países` (`idestado`)
                        ON DELETE NO ACTION
                        ON UPDATE NO ACTION); """   

        try:
            
            mycursor.execute(query1)
            mycursor.execute(query2)
            mydb.commit()
            print("Las tablas de la base de datos se han podido crear correctamente.")

        except mysql.connector.Error as err:

            print("No se han podido crear las tablas de la base de datos.")
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg) 

    def cargar_provincias(self, dataframe):

        df_provincias = dataframe[["country", "state_province", "lat", "long"]].drop_duplicates()

        mydb = mysql.connector.connect(user="root",
                                    password= f"{self.contraseña}",
                                    host="localhost", 
                                    database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()
    
        for indice, fila in df_provincias.iterrows():

            try:

                check_provincia = f"""SELECT idestado FROM países 
                                    WHERE nombre_provincia = "{fila['state_province']}" AND nombre_país = "{fila['country']}";"""
                mycursor.execute(check_provincia)
                existe_provincia = mycursor.fetchone()
            
                if not existe_provincia:

                    query_provincia = f"""INSERT INTO países (nombre_país, nombre_provincia, latitud, longitud) 
                                            VALUES ("{fila['country']}", "{fila['state_province']}", 
                                            {'NULL' if pd.isna(fila['lat']) else fila['lat']},
                                            {'NULL' if pd.isna(fila['long']) else fila['long']});
                                            
                                        """
                    
                    mycursor.execute(query_provincia)
                    mydb.commit()

                else:
                    print(f"Ya existe la provincia {fila['state_province']} en la base de datos.")

            
            except mysql.connector.Error as err:

                print("No se han podido cargar las provincias a la base de datos.")
                print(err)
                print("Error Code:", err.errno)
                print("SQLSTATE", err.sqlstate)
                print("Message", err.msg)          


    def cargar_universidades(self, dataframe):


        df_universidad = dataframe[["name", "web_pages", "country", "state_province"]].drop_duplicates()

        mydb = mysql.connector.connect(user="root",
                                    password= f"{self.contraseña}",
                                    host="localhost", 
                                    database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()
        
        for indice, fila in df_universidad.iterrows():

            try:

                check_universidad = f"""SELECT id_universidad FROM universidades 
                                        WHERE nombre_universidad = "{fila['name']}"; 
                                        """
                mycursor.execute(check_universidad)
                existe_universidad = mycursor.fetchone()
                    
                if not existe_universidad:

                    try:

                        check_provincia = f"""SELECT idestado FROM países 
                                            WHERE nombre_provincia = "{fila['state_province']}" AND nombre_país = "{fila['country']}";"""
                                            
                        mycursor.execute(check_provincia)

                        id_estado = mycursor.fetchone()[0]
        
                        query_universidad = f"""INSERT INTO universidades (nombre_universidad, pagina_web, países_idestado) 
                                                VALUES ("{fila['name']}", "{fila['web_pages']}", "{id_estado}");
                                                """
                                
                        mycursor.execute(query_universidad)
                        mydb.commit()
                    
                    except mysql.connector.Error as err:

                        print("No se han podido cargar las provincias a la base de datos.")
                        print(err)
                        print("Error Code:", err.errno)
                        print("SQLSTATE", err.sqlstate)
                        print("Message", err.msg)     

                else:

                    print(f"Ya existe la universidad {fila['name']} en la base de datos.")

                
            except mysql.connector.Error as err:

                print("No se han podido cargar las provincias a la base de datos.")
                print(err)
                print("Error Code:", err.errno)
                print("SQLSTATE", err.sqlstate)
                print("Message", err.msg)       


In [60]:
df_uni.head(1)

,alpha_two_code,state_province,country,name,web_pages,lat,long
0,AR,Buenos Aires,Argentina,Universidad Atlantida Argentina,http://www.atlantida.edu.ar/,-34.607568,-58.437089


In [61]:
bbdd = Cargar("universidades", "AlumnaAdalab")

In [62]:
bbdd.crear_bbdd()

La conexión se ha realizado con éxito.
La base de datos universidades se ha creado con éxito o ya existía anteriormente.


In [63]:
bbdd.crear_tablas()

Las tablas de la base de datos se han podido crear correctamente.


In [64]:
df_uni

,alpha_two_code,state_province,country,name,web_pages,lat,long
0,AR,Buenos Aires,Argentina,Universidad Atlantida Argentina,http://www.atlantida.edu.ar/,-34.607568,-58.437089
1,AR,Buenos Aires,Argentina,Universidad Austral Buenos Aires,http://www.austral.edu.ar/,-34.607568,-58.437089
2,AR,Buenos Aires,Argentina,"Universidad CAECE, Buenos Aires",http://www.caece.edu.ar/,-34.607568,-58.437089
3,AR,Buenos Aires,Argentina,Instituto Universitario CEMA,http://www.cema.edu.ar/,-34.607568,-58.437089
4,AR,Buenos Aires,Argentina,Instituto de Enseñanza Superior del Ejército,http://www.iese.edu.ar/,-34.607568,-58.437089
...,...,...,...,...,...,...,...
2486,US,Unknown,United States,Chicago State University,https://csu.edu,NaN,NaN
2487,US,Unknown,United States,Colorado State University-Pueblo,https://csupueblo.edu,NaN,NaN
2488,US,Unknown,United States,Bay Atlantic University,https://bau.edu/,NaN,NaN
2489,US,Unknown,United States,Siena College,https://www.siena.edu/,NaN,NaN


In [65]:
bbdd.cargar_provincias(df_uni)

In [66]:
df_uni["lat"].loc[0]


-34.6075682

In [67]:
bbdd.cargar_universidades(df_uni)

No se han podido cargar las provincias a la base de datos.
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'John F. Kennedy""' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'John F. Kennedy""' at line 2
No se han podido cargar las provincias a la base de datos.
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Juan Agustín Maza""' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Juan Agustín Maza""' at line 2
No se han podido cargar las provincias a la base de datos.
1064 (42000): You have an error in your SQL syntax; che

In [68]:
# No ha introducido todos los registros, ya que sólo hya 2486 registros en la BBDD, mientras que hay un total de 2491 aquí.

df_uni.shape

(2491, 7)